## Imports:

In [1]:
from __future__ import unicode_literals
from __future__ import division
from __future__ import print_function

import os

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 60)
# import tables as h5

import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd

xls_writer = pd.ExcelWriter

import pybedtools as pbt

import scipy
import statsmodels.api as sm
import statsmodels.stats.multitest as smm

import munch

from spartan.utils.genome_specific.GfusI1 import GfusI1_0
from spartan.utils.fastas import ParseFastA


## File paths:

In [2]:
# define paths to files

tx_fasta = "/home/gus/remote_mounts/louise/data/genomes/glossina_morsitans/cufflinks_transcripts.fa"

ortholog_table = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/Find_Gff_sequences_from_Gmm_RNA-seq_transcriptome/existing_genes/vectorbase_orthologs_GMOY_GFUI.csv"

tx_orthos_out = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/Find_Gff_sequences_from_Gmm_RNA-seq_transcriptome/existing_genes/xiaoqing_orthologous_Tx_GMOY_GFUI.xls"

In [14]:
tx_names = ParseFastA(tx_fasta, key=lambda h: tuple(h.lstrip('>').split(' gene='))).to_dict().keys()

In [18]:
tx_df = pd.DataFrame(tx_names, columns=['tx_tcons','tx_gene'])
tx_df.head()

,tx_tcons,tx_gene
0,TCONS_00022941,GMOY008319
1,TCONS_00025678,GMOY009334
2,TCONS_00011714,GMOY004150
3,TCONS_00012654,GMOY004487
4,TCONS_00032104,GMOY011517


In [20]:
orthos = pd.read_csv(ortholog_table)
orthos.head()

,Gene stable ID,Glossina fuscipes fuscipes gene stable ID,Homology type,% identity,Glossina fuscipes fuscipes % identity,dN,dS,"Orthology confidence [0 low, 1 high]"
0,GMOY005425,GFUI009881,ortholog_one2one,90,47,0.0635,0.2015,1
1,GMOY005426,GFUI009872,ortholog_one2one,84,91,0.0402,0.1808,1
2,GMOY005427,GFUI009879,ortholog_one2one,79,45,0.1107,0.1997,1
3,GMOY005428,GFUI009867,ortholog_one2one,87,91,0.0477,0.1389,1
4,GMOY005430,GFUI009870,ortholog_one2one,99,98,0.0049,0.2207,1


In [21]:
tx_orthos = pd.merge(left=tx_df, right=orthos, 
                     how='inner', on=None, 
                     left_on='tx_gene', right_on='Gene stable ID', 
                     left_index=False, right_index=False, 
                     sort=False, suffixes=('_x', '_y'), 
                     copy=True, indicator=False)
tx_orthos.head()

,tx_tcons,tx_gene,Gene stable ID,Glossina fuscipes fuscipes gene stable ID,Homology type,% identity,Glossina fuscipes fuscipes % identity,dN,dS,"Orthology confidence [0 low, 1 high]"
0,TCONS_00022941,GMOY008319,GMOY008319,GFUI001502,ortholog_one2one,97,97,0.0123,0.1856,1
1,TCONS_00022942,GMOY008319,GMOY008319,GFUI001502,ortholog_one2one,97,97,0.0123,0.1856,1
2,TCONS_00022939,GMOY008319,GMOY008319,GFUI001502,ortholog_one2one,97,97,0.0123,0.1856,1
3,TCONS_00022940,GMOY008319,GMOY008319,GFUI001502,ortholog_one2one,97,97,0.0123,0.1856,1
4,TCONS_00011714,GMOY004150,GMOY004150,GFUI006585,ortholog_one2one,95,92,0.0298,0.1396,1


In [22]:
tx_orthos.shape

(23803, 10)

In [24]:
# unique GENES 
tx_orthos.tx_gene.unique().shape

(9948,)

In [25]:
# unique TX 
tx_orthos.tx_tcons.unique().shape

(21900,)

In [29]:
tx_orthos.drop("tx_gene", axis=1).to_excel(tx_orthos_out,index=False)